<a href="https://www.kaggle.com/code/gpreda/romanian-llm?scriptVersionId=185412627" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

**OpenLLM-Ro** is a new initiative to develop a Romanian LLM, based on existing large language models. The first version is based on Llama2, fine-tuned with a corpus of Romanian language sources data.

For more details about the project, check this link: https://ilds.ro/llm-for-romanian/


# Load the model

We are loading for now the model from HuggingFace. In the future, we might migrate it to Kaggle, as a Kaggle Model.

In [1]:
from time import time
from IPython.display import display, Markdown
from transformers import pipeline, AutoConfig, AutoModelForCausalLM, AutoTokenizer

2024-06-25 16:50:54.701216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 16:50:54.701338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 16:50:54.884845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model_id = "OpenLLM-Ro/RoLlama2-7b-Instruct-v1"
model_config = AutoConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prepare query function

In [4]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipe(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipe.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
"""

In [5]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Test with few simple general information questions

Because this is a Romanian LLM, we will ask the questions in Romanian.

## Utility function to display the results of the query

In [6]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [7]:
response = query_model(
    system_message,
    user_message="Care este suprafata Romaniei?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.





**<font color='red'>Question:</font>** Care este suprafata Romaniei? 

**<font color='green'>Answer:</font>**  Suprafața României este de 238.391 km².  

**<font color='magenta'>Total time:</font>** 5.36 sec.

In [8]:
response = query_model(
    system_message,
    user_message="Cine a fost Stefan cel Mare?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine a fost Stefan cel Mare? 

**<font color='green'>Answer:</font>**  Ștefan cel Mare a fost un domnitor al Moldovei din 1457 până în 1504. El este considerat unul dintre cei mai mari conducători ai Europei medievale și este cunoscut pentru conducerea sa militară și politică, precum și pentru realizările sale în dezvoltarea statului Moldova.  

**<font color='magenta'>Total time:</font>** 9.84 sec.

In [9]:
response = query_model(
    system_message,
    user_message="In ce an a castigat faima mondiala Nadia Comaneci?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** In ce an a castigat faima mondiala Nadia Comaneci? 

**<font color='green'>Answer:</font>**  Nadia Comăneci a câștigat faima mondială în 1976, când a devenit prima gimnastă care a primit nota perfectă de zece la Jocurile Olimpice de la Montreal.  

**<font color='magenta'>Total time:</font>** 6.55 sec.

In [10]:
response = query_model(
    system_message,
    user_message="Poti numi cateva carti ale lui Mihail Sadoveanu?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Poti numi cateva carti ale lui Mihail Sadoveanu? 

**<font color='green'>Answer:</font>**  Da, pot numi câteva cărți ale lui Mihail Sadoveanu:

1. „Baltagul” (1930)
2. „Hanu Ancutei” (1929)
3. „Ciuleandra” (1 

**<font color='magenta'>Total time:</font>** 7.07 sec.

In [11]:
response = query_model(
    system_message,
    user_message="In ce an a castigat Steaua Bucuresti Cupa Campionilor Europeni?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** In ce an a castigat Steaua Bucuresti Cupa Campionilor Europeni? 

**<font color='green'>Answer:</font>**  Steaua București a câștigat Cupa Campionilor Europeni în 1986.  

**<font color='magenta'>Total time:</font>** 3.21 sec.

In [12]:
response = query_model(
    system_message,
    user_message="Numeste cateva parcuri din Bucuresti.",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Numeste cateva parcuri din Bucuresti. 

**<font color='green'>Answer:</font>**  Parcurile din București sunt:
1. Herăstrău
2. Cișmigiu
3. Carol
4. Tineretului
5. Floreasca
6. Kiseleff
7. Regele Mihai I
8. Alexandru Io 

**<font color='magenta'>Total time:</font>** 7.08 sec.

In [13]:
response = query_model(
    system_message,
    user_message="Care este cel mai inalt varf din Romania?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Care este cel mai inalt varf din Romania? 

**<font color='green'>Answer:</font>**  Cel mai înalt vârf din România este Muntele Moldoveanu, care are o altitudine de 2.544 metri (8.346 picioare).  

**<font color='magenta'>Total time:</font>** 4.91 sec.

In [14]:
response = query_model(
    system_message,
    user_message="In ce tari au emigrat romanii intre 2004 si 2020?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** In ce tari au emigrat romanii intre 2004 si 2020? 

**<font color='green'>Answer:</font>**  Romanii au emigrat în principal în țări precum Italia, Spania, Germania, Regatul Unit, Franța, Republica Moldova, Canada, Statele Unite și Republica Irlanda între 2004 și 2020.  

**<font color='magenta'>Total time:</font>** 6.79 sec.

In [15]:
response = query_model(
    system_message,
    user_message="Cine este actualul presedinte al Romaniei?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine este actualul presedinte al Romaniei? 

**<font color='green'>Answer:</font>**  Actualul președinte al României este Klaus Iohannis.  

**<font color='magenta'>Total time:</font>** 2.34 sec.

In [16]:
response = query_model(
    system_message,
    user_message="Ce europarlamentari romani fac parte din Renew Europe?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Ce europarlamentari romani fac parte din Renew Europe? 

**<font color='green'>Answer:</font>**  Europarlamentarii români care fac parte din Renew Europe sunt Ramona Strugariu, Dragoș Tudorache și Dragoș Pîslaru.  

**<font color='magenta'>Total time:</font>** 4.6 sec.

In [17]:
response = query_model(
    system_message,
    user_message="Care este sectorul cel mai populat dintre cele 6 sectoare din Bucuresti?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104



**<font color='red'>Question:</font>** Care este sectorul cel mai populat dintre cele 6 sectoare din Bucuresti? 

**<font color='green'>Answer:</font>**  Sectorul 6 este cel mai populat dintre cele 6 sectoare din București.  

**<font color='magenta'>Total time:</font>** 3.01 sec.

# Simple math questions

In [18]:
system_message = """
Esti un AI Agent specializat sa raspunda la intrebari de matematica in limba romana.
"""
response = query_model(
    system_message,
    user_message="Care este suprafata unui cerc de raza 3?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Care este suprafata unui cerc de raza 3? 

**<font color='green'>Answer:</font>**  Suprafața unui cerc cu raza 3 este de 28,27 unități pătrate.  

**<font color='magenta'>Total time:</font>** 3.54 sec.

In [19]:
response = query_model(
    system_message,
    user_message="""Ana si Dragos au impreuna 6 mere. 
    Dragos are cu 2 mere mai mult ca Ana. 
    Cate mere are fiecare dintre cei doi copii?""",
    temperature=0.1,
    max_length=128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Ana si Dragos au impreuna 6 mere. 
    Dragos are cu 2 mere mai mult ca Ana. 
    Cate mere are fiecare dintre cei doi copii? 

**<font color='green'>Answer:</font>**  Ana are 4 mere, iar Dragos are 6 mere.  

**<font color='magenta'>Total time:</font>** 2.23 sec.

# Conclusions

The Romanian Large Language Model (Romanian LLM: **OpenLLM-Ro**) is able to answer correctly to general information questions in Romanian, including recent events.

Tested with 2 simple math questions, was able to answer approximately correct, to the geometry question correct, and to an algebra question with an error.